# Data import

data를 가져오는 부분

In [18]:
import csv
import numpy as np

with open('train.csv','r') as dest_f:
    data_iter = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data = [data for data in data_iter]
data_array = np.asarray(data, dtype = 'str')
print(data_array)
#print(data_array.shape)

[['PassengerId' 'Survived' 'Pclass' ... 'Fare' 'Cabin' 'Embarked']
 ['1' '0' '3' ... '7.25' '' 'S']
 ['2' '1' '1' ... '71.2833' 'C85' 'C']
 ...
 ['889' '0' '3' ... '23.45' '' 'S']
 ['890' '1' '1' ... '30' 'C148' 'C']
 ['891' '0' '3' ... '7.75' '' 'Q']]


data의 한글부분을 수치화한다.
(남자는 0, 여자는 1로)

In [19]:
quantifi_data = data_array[1:,:]

for i in range(quantifi_data.shape[0]):
    if quantifi_data[i][4]=='male':
        quantifi_data[i][4]=float(0)
    elif quantifi_data[i][4]=='female':
        quantifi_data[i][4]=float(1)

### 나이 데이터의 빈 공간을 채우는 방법을 찾아보자

방법 1. 1912년 미국의 기대수명을 이용한다. 남자의 경우 51.5살, 여자의 경우 55.9살이라는 데이터를 이용한다.
###### http://www.demog.berkeley.edu/~andrew/1918/figure2.html

In [20]:
for i in range(quantifi_data.shape[0]):
    if quantifi_data[i][5]=='':
        if quantifi_data[i][4]:
            quantifi_data[i][5]=float(51.5)
        else:
            quantifi_data[i][5]=float(55.9)
            
print(quantifi_data[5])

['6' '0' '3' 'Moran, Mr. James' '0.0' '51.5' '0' '0' '330877' '8.4583' ''
 'Q']


test 할 데이터 또한 받아온후 동일 작업을 시행한다

In [21]:
with open('test.csv','r') as dest_f:
    data_iter1 = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data1 = [data1 for data1 in data_iter1]
data_array1 = np.asarray(data1, dtype = 'str')
print(data_array1)
#print(data_array.shape)

quantifi_data1 = data_array1[1:,:]

for i in range(quantifi_data1.shape[0]):
    if quantifi_data1[i][3]=='male':
        quantifi_data1[i][3]=0
    elif quantifi_data1[i][3]=='female':
        quantifi_data1[i][3]=1
        
for i in range(quantifi_data1.shape[0]):
    if quantifi_data1[i][4]=='':
        if quantifi_data1[i][3]:
            quantifi_data1[i][4]=float(51.5)
        else:
            quantifi_data1[i][4]=float(55.9)

[['PassengerId' 'Pclass' 'Name' ... 'Fare' 'Cabin' 'Embarked']
 ['892' '3' 'Kelly, Mr. James' ... '7.8292' '' 'Q']
 ['893' '3' 'Wilkes, Mrs. James (Ellen Needs)' ... '7' '' 'S']
 ...
 ['1307' '3' 'Saether, Mr. Simon Sivertsen' ... '7.25' '' 'S']
 ['1308' '3' 'Ware, Mr. Frederick' ... '8.05' '' 'S']
 ['1309' '3' 'Peter, Master. Michael J' ... '22.3583' '' 'C']]


#### 성별과 동승자수, Pclass, Parch, 나이로 분류해보자
#### x data size는 [datanum, 5(Pclass, 성별, 나이, 동승자수, Parch)]
#### y data size는 [datanum, 1]이다

In [22]:
import tensorflow as tf

x_data = np.vstack([quantifi_data[:,2], quantifi_data[:,4], quantifi_data[:,5], quantifi_data[:,6], quantifi_data[:,7]]).T
y_data = quantifi_data[:,1].reshape(quantifi_data.shape[0],1)
x_test_data = np.vstack([quantifi_data1[:,1], quantifi_data1[:,3], quantifi_data1[:,4], quantifi_data1[:,5], quantifi_data1[:,6]]).T
test_prediction = np.zeros([quantifi_data1.shape[0],1])

X = tf.placeholder(tf.float32, [None, 5])
Y = tf.placeholder(tf.float32, [None, 1])

W1 = tf.Variable(tf.random_normal([5,5]), name='weight1')
b1 = tf.Variable(tf.random_normal([5]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([5,5]), name='weight2')
b2 = tf.Variable(tf.random_normal([5]), name='bias2')
layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([5,3]), name='weight3')
b3 = tf.Variable(tf.random_normal([3]), name='bias3')
layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)

W4 = tf.Variable(tf.random_normal([3,1]), name='weight4')
b4 = tf.Variable(tf.random_normal([1]), name='bias4')
hypothesis = tf.sigmoid(tf.matmul(layer3, W4) + b4)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
#train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(20001):
        sess.run(train, feed_dict={X: x_data,Y: y_data})
        
        if step % 1000 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data,Y: y_data}), sess.run([W1, W2, W3]))
    
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data,Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)
    
    #이제 실제 데이터를 집어넣어 예측해보자
    test_prediction = sess.run([hypothesis], feed_dict={X: x_test_data})
    print(test_prediction)
    

0 0.6626336 [array([[-0.27637345, -0.39232886,  0.4074656 ,  0.20529446, -0.0789383 ],
       [ 1.0432361 , -1.951263  , -0.40338653,  0.7710586 ,  0.7697624 ],
       [ 2.0875006 ,  0.10513677,  0.9601078 ,  0.2633627 ,  0.9597265 ],
       [ 1.1223421 ,  0.17727473, -0.5051194 ,  2.1215806 ,  1.0914832 ],
       [-0.8618762 , -0.9778477 ,  0.52410984, -1.1350392 , -2.6998203 ]],
      dtype=float32), array([[-0.6784854 , -0.91605985,  1.077252  , -0.46186256, -0.6886491 ],
       [-0.42342064, -1.3592414 ,  0.14592075,  0.09887163,  1.878551  ],
       [ 0.5885836 ,  0.10356633, -0.33273107,  2.0636256 ,  0.8662995 ],
       [-1.3064947 , -0.23643488,  0.19821882, -0.5515266 ,  0.7135862 ],
       [ 0.8992219 , -0.9163638 ,  0.40540746,  1.419324  , -0.15807188]],
      dtype=float32), array([[ 0.17610544,  0.12678605,  0.6975543 ],
       [ 0.21336228,  0.47023517, -0.94021755],
       [-0.35355312, -0.8386442 ,  0.0766008 ],
       [-1.0123217 ,  1.814054  , -1.8624914 ],
       [-

8000 0.3853078 [array([[  2.4891346 ,   4.781658  ,  -0.46799427,  -6.7565656 ,
        -10.164938  ],
       [  2.0174663 ,  -9.704436  , -10.756868  , -11.015353  ,
          7.3719535 ],
       [  4.1364675 ,  -0.05243096,   0.37156382,   1.9122185 ,
          6.17654   ],
       [  2.2861931 ,   0.7067271 , -13.56243   ,  11.166202  ,
         18.370707  ],
       [  1.4296782 ,   0.42772332,   4.926658  ,  -7.7670937 ,
         -4.3782134 ]], dtype=float32), array([[  2.1339166,   2.9670215,   4.2960052,  -3.409811 ,  -4.2788258],
       [-67.708405 ,  -5.5406017,   6.3596377,   5.135607 ,  12.98442  ],
       [  0.8218099, -62.65441  ,  -2.100305 ,   6.7094946,  -8.242936 ],
       [ -9.08429  ,  -7.112218 ,   3.3375576, -11.039888 ,  -5.8216243],
       [  1.8673989,  -3.0575497,   3.7794414,   9.265946 ,  -2.669958 ]],
      dtype=float32), array([[  6.342192 , -15.193633 ,  34.25016  ],
       [ 19.996187 , -10.858109 ,  -6.523625 ],
       [ -0.8868269,  -1.3027993,  -4.21033

15000 0.37670377 [array([[  4.428954  ,   4.9927535 ,   0.11418176,  -5.2594    ,
        -12.652012  ],
       [  4.263593  ,  -9.790824  , -12.847726  , -18.772408  ,
         12.880415  ],
       [  4.9956203 ,  -0.05921897,   0.4178935 ,   2.5081525 ,
          6.1513805 ],
       [  1.1735348 ,   0.5616333 , -15.005871  ,  15.941222  ,
         19.127653  ],
       [  1.1308987 ,   0.56837505,   5.3767333 , -13.443337  ,
          0.6525796 ]], dtype=float32), array([[   3.1264923,    3.0535855,    2.9541285,   -4.666529 ,
          -6.5494537],
       [-131.0343   ,   -5.684192 ,    6.2698636,    7.8385787,
          18.076279 ],
       [   1.6194263, -153.03351  ,   -2.0421503,   11.07657  ,
         -16.416027 ],
       [ -10.321496 ,   -6.659949 ,    3.5105658,  -12.078997 ,
         -17.561308 ],
       [   1.8898323,   -4.484865 ,    3.743367 ,   10.029993 ,
          -3.311737 ]], dtype=float32), array([[ 16.54494   , -32.404346  ,  50.05947   ],
       [ 38.741535  , -18.4

KeyboardInterrupt: 

test_prediction의 값을 0과 1로 정규화 시켜준다

In [6]:
for i in range(len(test_prediction[0])):
    if test_prediction[0][i] > 0.5:
        test_prediction[0][i] = 1
    else:
        test_prediction[0][i] = 0

print(test_prediction)

[array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.]

예측값들을 csv파일로 저장한다

In [7]:
f = open('prediction4.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
wr.writerow(["PassengerId", "Survived"])
for i in range(len(test_prediction[0])):
    wr.writerow([892+i, int(test_prediction[0][i])])
    
f.close()